In [ ]:
#@title 📦 Batch LLM Processor (Folders-as-Batches, Hidden Code, Drive Upload, Random Subset)
#@markdown **What’s new:** If your ZIP contains **subfolders**, each subfolder is processed as **one payload** (all files inside it are sent together).  
#@markdown - Root-level files (not in any folder) are processed **one-by-one**.  
#@markdown - A folder can contain a **mix of text + images**; they will be sent together in one request.  
#@markdown **Note:** Big folders (many/large images) can exceed model or API limits.

#@markdown ---
#@markdown **Instructions:**  
#@markdown 1. Enter your **OpenRouter API Key**.  
#@markdown 2. Enter a **System Prompt** (optional).  
#@markdown 3. Choose or type a **Model ID**.  
#@markdown 4. Provide your ZIP via **Upload** or **Google Drive path**.  
#@markdown 5. (Optional) Tick the box to include inputs in output ZIP.  
#@markdown 6. (Optional) Limit how many **items** are processed (items = folders + root files). Selection is random with a seed.  
#@markdown 7. Click **Run Processing** and download results.

import os, zipfile, base64, mimetypes, json, time, random
import pandas as pd
import requests
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files as colab_files, drive

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"

# --- UI Controls ---
api_key = widgets.Password(description="API Key:", layout=widgets.Layout(width="420px"))
system_prompt = widgets.Textarea(
    description="System Prompt:",
    layout=widgets.Layout(width="650px", height="90px"),
    placeholder="e.g., For each item, return a short description and tags."
)

model_dropdown = widgets.Dropdown(
    options=[
        ("Gemini 2.5 Flash (default)", "google/gemini-2.5-flash"),
        ("Gemini 2.5 Pro", "google/gemini-2.5-pro"),
        ("GPT-5", "openai/gpt-5"),
        ("GPT-5 Mini", "openai/gpt-5-mini"),
        ("GPT-4.1", "openai/gpt-4.1"),
        ("Claude Sonnet", "anthropic/claude-sonnet-4"),
        ("Claude Opus", "anthropic/claude-opus-4.1"),
        ("Mistral Medium 3.1", "mistralai/mistral-medium-3.1"),
        ("Grok 4", "x-ai/grok-4"),
        ("Grok 4 Fast", "x-ai/grok-4-fast"),
    ],
    value="google/gemini-2.5-flash",
    description="Model:"
)

model_custom = widgets.Text(
    description="Custom Model:",
    placeholder="e.g. openai/gpt-4.1-mini",
    layout=widgets.Layout(width="520px")
)

include_inputs = widgets.Checkbox(value=False, description="Include inputs in output zip")

upload_method = widgets.RadioButtons(
    options=["Upload (<100MB)", "Google Drive path"],
    description="Input ZIP:"
)

upload_btn = widgets.FileUpload(accept=".zip", multiple=False, description="Upload ZIP")
drive_path = widgets.Text(
    description="Drive path:",
    placeholder="/content/drive/MyDrive/myfile.zip",
    layout=widgets.Layout(width="650px")
)

max_items = widgets.IntText(value=0, description="Max items (0=all):")
seed_value = widgets.IntText(value=2025, description="Seed:")

run_btn = widgets.Button(description="Run Processing", button_style="success")

ui_box = widgets.VBox([
    api_key,
    system_prompt,
    model_dropdown,
    model_custom,
    include_inputs,
    upload_method,
    upload_btn,
    drive_path,
    widgets.HBox([max_items, seed_value]),
    run_btn
])
display(ui_box)


def _safe_list_zip_files(zf: zipfile.ZipFile):
    """Return a list of file entries in the zip (no directories), skipping common junk."""
    names = []
    for n in zf.namelist():
        if not n or n.endswith("/"):
            continue
        # skip macOS resource forks / folders
        if n.startswith("__MACOSX/") or os.path.basename(n).startswith("._"):
            continue
        names.append(n)
    return names


def _read_text_file(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except UnicodeDecodeError:
        with open(path, "r", encoding="latin-1") as f:
            return f.read()


def _build_groups(file_list):
    """Group files by folder. Root files each become their own group."""
    groups = {}
    for rel in file_list:
        rel = rel.replace("\\", "/")
        folder = os.path.dirname(rel)
        if folder == "":
            key = rel  # root-level file -> its own group
        else:
            key = folder  # folder group
        groups.setdefault(key, []).append(rel)
    # stable ordering
    for k in groups:
        groups[k] = sorted(groups[k])
    return groups


def _make_user_content_for_group(group_key: str, extracted_input_dir: str, members: list):
    """Create OpenRouter multimodal user content for a group (folder or single root file)."""
    user_content = []
    unsupported = []

    # Add a small header so the model knows it's a batch
    user_content.append({
        "type": "text",
        "text": f"Batch item: {group_key}\nFiles included: {len(members)}\n"
    })

    for rel_name in members:
        fpath = os.path.join(extracted_input_dir, rel_name)
        ext = os.path.splitext(rel_name)[1].lower()

        if ext in [".txt", ".md"]:
            text = _read_text_file(fpath)
            # include filename marker to keep texts distinct
            user_content.append({"type": "text", "text": f"--- FILE: {rel_name} ---\n{text}"})
        elif ext in [".jpg", ".jpeg", ".png", ".tif", ".tiff"]:
            mime, _ = mimetypes.guess_type(fpath)
            if mime is None:
                mime = "image/png"
            with open(fpath, "rb") as img_file:
                img_b64 = base64.b64encode(img_file.read()).decode("utf-8")
            user_content.append({"type": "text", "text": f"Image file: {rel_name}"})
            user_content.append({"type": "image_url", "image_url": {"url": f"data:{mime};base64,{img_b64}"}})
        else:
            unsupported.append(rel_name)

    if unsupported:
        user_content.append({
            "type": "text",
            "text": "Unsupported files (ignored):\n" + "\n".join(unsupported)
        })

    return user_content


def process_zip(api_key_str: str, system_prompt_str: str, model: str, zip_path: str,
                include_inputs_flag: bool, max_items_n: int, seed_n: int) -> str:
    job_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    work_dir = f"job_{job_id}"
    os.makedirs(work_dir, exist_ok=True)

    # Extract selected entries here (preserving subfolders)
    input_dir = os.path.join(work_dir, "input")
    os.makedirs(input_dir, exist_ok=True)

    # 1) Read zip file list and build groups (folders = one group, root files = one group each)
    with zipfile.ZipFile(zip_path, "r") as zf:
        all_files = _safe_list_zip_files(zf)
        if not all_files:
            raise RuntimeError("ZIP contains no files (or only folders).")

        groups = _build_groups(all_files)
        group_keys = sorted(groups.keys())  # deterministic before sampling

        # Random sampling of *groups/items* if requested
        if max_items_n and max_items_n > 0 and max_items_n < len(group_keys):
            random.seed(seed_n)
            selected_keys = sorted(random.sample(group_keys, max_items_n))
        else:
            selected_keys = group_keys

        # Extract only the files needed for selected groups
        selected_files = []
        for gk in selected_keys:
            selected_files.extend(groups[gk])
        # de-dup (in case of edge cases)
        selected_files = sorted(set(selected_files))

        for member in selected_files:
            zf.extract(member, input_dir)

    # 2) Process each group as ONE payload
    rows = []
    total = len(selected_keys)

    for idx, group_key in enumerate(selected_keys, start=1):
        members = groups[group_key]  # list of rel paths (inside zip)
        # Build multimodal content containing all files in the group
        user_content = _make_user_content_for_group(group_key, input_dir, members)

        payload = {
            "model": model,
            "messages": [
                {"role": "system", "content": system_prompt_str},
                {"role": "user", "content": user_content}
            ]
        }
        headers = {"Authorization": f"Bearer {api_key_str}"}

        try:
            r = requests.post(OPENROUTER_URL, json=payload, headers=headers, timeout=240)
            r.raise_for_status()
            data = r.json()
            reply = data["choices"][0]["message"]["content"]
        except Exception as e:
            reply = f"ERROR: {e}"

        rows.append({"item": group_key, "files_in_item": len(members), "output": reply})
        clear_output(wait=True)
        print(f"Processed {idx}/{total}: {group_key} (files: {len(members)})")
        time.sleep(0.2)

    # 3) Save outputs
    output_csv = os.path.join(work_dir, "output.csv")
    pd.DataFrame(rows).to_csv(output_csv, index=False)

    # input.csv should reflect the processed groups and their members
    input_rows = []
    for group_key in selected_keys:
        for rel_name in groups[group_key]:
            fpath = os.path.join(input_dir, rel_name)
            try:
                fsize = os.path.getsize(fpath)
            except OSError:
                fsize = None
            ftype = mimetypes.guess_type(fpath)[0] or "unknown"
            input_rows.append({
                "item": group_key,
                "file_name": rel_name,
                "file_type": ftype,
                "file_size": fsize
            })

    input_csv = os.path.join(work_dir, "input.csv")
    pd.DataFrame(input_rows).to_csv(input_csv, index=False)

    meta = {
        "model": model,
        "system_prompt": system_prompt_str,
        "submitted_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "include_inputs": include_inputs_flag,
        "max_items": int(max_items_n) if max_items_n is not None else 0,
        "seed": int(seed_n) if seed_n is not None else 2025,
        "total_files_in_zip": len(all_files),
        "total_items_in_zip": len(groups),
        "processed_items": len(selected_keys),
        "processed_files": len(input_rows)
    }
    meta_path = os.path.join(work_dir, "meta.json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2, ensure_ascii=False)

    # 4) Create results zip (include only selected inputs if requested)
    zip_out = f"results_{job_id}.zip"
    with zipfile.ZipFile(zip_out, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.write(output_csv, arcname="output.csv")
        zf.write(input_csv, arcname="input.csv")
        zf.write(meta_path, arcname="meta.json")

        if include_inputs_flag:
            # include ONLY selected files (folders preserved)
            for rel_name in selected_files:
                fpath = os.path.join(input_dir, rel_name)
                if os.path.isfile(fpath):
                    zf.write(fpath, arcname=os.path.join("input", rel_name))

    return zip_out


def on_run_clicked(_):
    if not api_key.value.strip():
        print("⚠️ Please enter your API Key.")
        return

    # Resolve zip path from upload or Drive
    if upload_method.value == "Upload (<100MB)":
        if not upload_btn.value:
            print("⚠️ Please upload a ZIP file first.")
            return
        fname = list(upload_btn.value.keys())[0]
        local_zip = "input_uploaded.zip"
        with open(local_zip, "wb") as f:
            f.write(upload_btn.value[fname]["content"])
        zip_path = local_zip
    else:
        zp = drive_path.value.strip()
        if not zp:
            print("⚠️ Please provide a Google Drive path.")
            return
        if not os.path.exists(zp):
            drive.mount("/content/drive")
        if not os.path.exists(zp):
            print(f"⚠️ File not found: {zp}")
            return
        zip_path = zp

    chosen_model = model_custom.value.strip() if model_custom.value.strip() else model_dropdown.value

    try:
        result_zip = process_zip(
            api_key_str=api_key.value.strip(),
            system_prompt_str=system_prompt.value,
            model=chosen_model,
            zip_path=zip_path,
            include_inputs_flag=include_inputs.value,
            max_items_n=max_items.value,
            seed_n=seed_value.value
        )
        print("✅ Processing complete. Download your results:")
        colab_files.download(result_zip)
    except Exception as e:
        print(f"❌ Error: {e}")

run_btn.on_click(on_run_clicked)
